In [1]:
from tqdm.notebook import tqdm
from IPython.display import display, HTML

import numpy as np
import pandas as pd

import plotly.graph_objects as go

import datetime
from pathlib import Path

import intake

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
import itertools

from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, RMSprop

2024-05-30 11:49:22.290379: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 11:49:23.368160: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
catalog = intake.open_catalog(Path('../catalog.yml'))
source = getattr(catalog, 'treesat_multi')
df = source.read()

In [3]:
selected_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12', 'TCI_R', 'TCI_G', 'TCI_B']

In [4]:
target = source.metadata['categories']['multi'] # multi / trinary
labels = df[target].to_numpy()

model_dir = Path('models').joinpath('seasons')
model_dir.mkdir(parents=True, exist_ok=True)

data_dir = Path('seasonal_median')

shuffled_indices = shuffle(np.arange(labels.shape[0]), random_state=42)

In [5]:
def get_features(years, seasons, data_dir, shuffled_indices):
    selected_data = []
    for year in years:
        for season in seasons:
            filepath = data_dir.joinpath(
                f'processed_treesat_{year}{str(season).zfill(2)}.npy')
            with open(filepath, 'rb') as f:
                data = np.load(f)
            selected_data.append(data[shuffled_indices])
    return np.stack(selected_data, axis=1)

In [10]:
class KerasModelCreator:
    def normalise_X(self, X, p=1):
        upper = np.percentile(X, 100-p)
        lower = np.percentile(X, p)
    
        bounded_X = np.where((X > upper) | (X < lower), np.median(X), X)

        scaled_X = (bounded_X - bounded_X.mean()) / bounded_X.std()
        return scaled_X

    def split_and_normalise(self, X, y):
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.1, shuffle=False)
                
        for i in range(X.shape[-1]):
            X_train[...,i] = self.normalise_X(X_train[...,i])
            X_test[...,i] = self.normalise_X(X_test[...,i])

        return X_train, X_test, y_train, y_test

    def display_logger(self, log_file, default_metrics):
        if log_file.is_file():
            val_metrics = ['val_' + x for x in default_metrics]
            df = pd.read_csv(log_file)[['epoch', 'loss'] + default_metrics + val_metrics]
            df['epoch'] += 1
            print('Previous training:')
            display(HTML(df.to_html(index=False)))


    def run(self, X, y, model_path, epochs=10, batch_size=10, overwrite=False):
        ''' 
        If not overwrite and there's an existing model, the model will 
        continue trainingw if the given epoch is bigger than the previous,
        else just evaluate.
        Ensure train splits are the same across continuations / evaluations
        by not modifying the random_state in split_and_normalise.
        '''
        model_savepoint = model_path.parent.joinpath(model_path.stem)
        log_file = model_path.with_suffix('.log')

        if overwrite:
            for f in [model_path, log_file] + list(model_savepoint.glob('*')):
                f.unlink(missing_ok=True)

        default_metrics = ['accuracy', 'root_mean_squared_error', 'r2_score']
                
        self.display_logger(log_file, default_metrics)
        
        callbacks = self.define_callbacks_and_logger(
            model_path, model_savepoint, log_file, default_metrics)
        
        X_train, X_test, y_train, y_test = self.split_and_normalise(X, y)
        
        if model_path.is_file():
            model = tf.keras.models.load_model(model_path)
        else:
            model = self.build_model(y.shape[1], X.shape[1:], default_metrics)
        
        model.fit(
            X_train, y_train, epochs=epochs, verbose=1, batch_size=batch_size, 
            callbacks=callbacks, shuffle=False, validation_data=(X_test, y_test)
        )

    def define_callbacks_and_logger(self, model_path, model_savepoint, log_file, default_metrics):    
        callbacks = [
            tf.keras.callbacks.BackupAndRestore(
                model_savepoint, save_freq='epoch', delete_checkpoint=False
            ),
            tf.keras.callbacks.CSVLogger(log_file, append=True),
            tf.keras.callbacks.ModelCheckpoint(
                model_path, monitor='val_accuracy', save_best_only=True, 
                save_freq='epoch', initial_value_threshold=0.4
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
              monitor='val_accuracy', factor=0.5, patience=2, min_lr=3e-6,
            ),
        ]
        return callbacks
        
    def build_model(self, output_shape, input_shape, metrics):
        m = tf.keras.Sequential()
        m.add(Input(input_shape))

        m.add(ConvLSTM2D(
            filters=16, kernel_size=2, 
            padding='same', 
            activation='tanh',
            kernel_regularizer=l2(0.001),
            return_sequences=False,
            unroll=True,
        ))
        m.add(MaxPooling2D(pool_size=2, padding='valid', strides=1))
        m.add(BatchNormalization())

        m.add(Conv2D(
            filters=32, kernel_size=2, padding='same', activation='relu',
        ))
        m.add(MaxPooling2D(pool_size=2, padding='valid', strides=1))
        m.add(BatchNormalization())

        m.add(Conv2D(
            filters=64, kernel_size=2, padding='same', activation='relu',
        ))
        m.add(MaxPooling2D(pool_size=2, padding='valid', strides=1))
        m.add(BatchNormalization())

        m.add(Flatten())
        m.add(Dense(128, activation='relu'))
        m.add(BatchNormalization())
    
        m.add(Dense(output_shape, activation='softmax'))

        adam = Adam(
            learning_rate=0.001,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-07,
            amsgrad=False,
        )

        m.compile(optimizer=adam, loss='mse', metrics=metrics)
        
        return m

In [11]:
train_years = [2017, 2018, 2019]
all_seasons = [3, 6, 9, 12]

model_name = f'lstm_seasons_{"_".join(map(str, all_seasons))}_years_{"_".join(map(str, train_years))}.keras'
model_path = model_dir.joinpath(model_name)

train_features = get_features(train_years, all_seasons, data_dir, shuffled_indices)
train_features = np.median(np.split(train_features, len(train_years), axis=1), axis=0)
train_features.shape

(50381, 4, 6, 6, 13)

In [ ]:
%%time
result = KerasModelCreator().run(
    train_features,
    labels[shuffled_indices],
    model_path, epochs=10, batch_size=10, overwrite=True
)

Epoch 1/10
2857/4535 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.4611 - loss: 0.0454 - r2_score: 0.0681 - root_mean_squared_error: 0.1924

In [9]:
test_years = [2020, 2021, 2022]
test_features = get_features(test_years, all_seasons, data_dir, shuffled_indices)
test_features = np.median(np.split(test_features, len(test_years), axis=1), axis=0)

for i in range(test_features.shape[-1]):
    test_features[...,i] = KerasModelCreator().normalise_X(test_features[...,i])
    
model = tf.keras.models.load_model(model_path)

model.evaluate(test_features, labels[shuffled_indices], return_dict=True)

ValueError: File not found: filepath=models/seasons/lstm_seasons_3_6_9_12_years_2017_2018_2019.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
tf.keras.models.load_model(model_path).summary()

In [ ]:
# import subprocess
# subprocess.run(['sudo', 'shutdown', 'now'])

In [ ]:
# season_combinations = itertools.chain.from_iterable(
#     itertools.combinations(all_seasons, r) for r in range(1, len(all_seasons)+1))
# season_combinations = list(season_combinations)